In [1]:
from dotenv import load_dotenv

from autogen.agentchat.contrib.agent_builder import AgentBuilder


load_dotenv()


builder = AgentBuilder(
    config_file_or_env="OAI_CONFIG_LIST",
    builder_model="gpt-4o-mini",
    agent_model="gpt-4o-mini",
)

In [2]:
from config import llm_config

building_task = "Explain black hole to 5 years old."
default_llm_config = {"config_list": llm_config["config_list"]}
agent_list, agent_configs = builder.build(building_task, default_llm_config, coding=True)

==> Generating agents...
['Space_Expert', 'Physics_Expert', 'Storytelling_Expert', 'Kids_Education_Expert', 'Astronomy_Expert'] are generated.
==> Generating system message...
Preparing system message for Space_Expert
Preparing system message for Physics_Expert
Preparing system message for Storytelling_Expert
Preparing system message for Kids_Education_Expert
Preparing system message for Astronomy_Expert
==> Generating description...
Preparing description for Space_Expert
Preparing description for Physics_Expert
Preparing description for Storytelling_Expert
Preparing description for Kids_Education_Expert
Preparing description for Astronomy_Expert
==> Creating agents...
Creating agent Space_Expert...
Creating agent Physics_Expert...
Creating agent Storytelling_Expert...
Creating agent Kids_Education_Expert...
Creating agent Astronomy_Expert...
Adding user console proxy...


In [3]:
from config import llm_config
import autogen

filter_dict = {"model": ["gpt-4o-mini"]}
llm_config["config_list"] = autogen.filter_config(llm_config["config_list"], filter_dict)

In [4]:
import autogen
from config import llm_config


def start_task(execution_task: str, agent_list: list):
    group_chat = autogen.GroupChat(
        agents=agent_list,
        messages=[],
        max_round=12,
    )
    manager = autogen.GroupChatManager(
        groupchat=group_chat,
        llm_config={"config_list": llm_config["config_list"]},
    )
    agent_list[0].initiate_chat(
        manager,
        message=execution_task,
    )


start_task(
    execution_task="Explain black hole to 5 years old.",
    agent_list=agent_list,
)

Space_Expert (to chat_manager):

Explain black hole to 5 years old.

--------------------------------------------------------------------------------



Next speaker: Kids_Education_Expert

Kids_Education_Expert (to chat_manager):

To explain a black hole to a 5-year-old, I would say:

"Hey there, kiddo! Imagine space like a big, dark room filled with cozy pillows. Now, think of a really strong vacuum cleaner that is super powerful—so powerful that it can suck in everything around it! 

That’s what a black hole is like. It’s a place in space that is made when a really big star gets tired and collapses in on itself. When it does that, it creates a spot that pulls everything nearby toward it, just like our super vacuum cleaner. 

But don’t worry! Black holes are super far away from us, so they can’t get us!

Now, imagine you have a magic ball that can show you things from far away. You see lots of stars dancing around the black hole, but if anything gets too close, 'whoosh!' it gets pulled in and disappears!

Isn't that cool? Do you have any questions about our super vacuum cleaner in space?" 

This way, we use relatable imagery and a b

InternalServerError: Error code: 500 - {'error': {'message': 'The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.', 'type': 'model_error', 'param': None, 'code': None}}

---


In [1]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from autogen import (
    AssistantAgent,
    UserProxyAgent,
    config_list_from_json,
    register_function,
)
from autogen.agentchat import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    register_function,
)
from autogen.agentchat.contrib import agent_builder
from autogen.agentchat.contrib.capabilities import teachability
from autogen.cache.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor
from config import llm_config
from dotenv import load_dotenv
from typing_extensions import Annotated

load_dotenv()

user_proxy = UserProxyAgent(
    name="Admin",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    system_message="A human admin. Give the task, and send instructions to writer to refine the blog post.",
    human_input_mode="NEVER",
    code_execution_config=False,
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Given a task, please determine what information is needed to complete the task.
Please note that the information will all be retrieved using Python code. Please only suggest information that can be retrieved using Python code. Reply TERMINATE when the task is done.
""",
    llm_config=llm_config,
)

engineer = AssistantAgent(
    name="Engineer",
    system_message="""Engineer. You write python/bash to retrieve relevant information. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
Reply TERMINATE when the task is done.
""",
    llm_config=llm_config,
)

writer = AssistantAgent(
    name="Writer",
    system_message="""Writer. Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You will write it for a task based on previous chat history. Don't write any code. Reply TERMINATE when the task is done.""",
    llm_config=llm_config,
)

os.makedirs("paper", exist_ok=True)
code_executor = UserProxyAgent(
    name="Executor",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    system_message="Executor. Execute the code written by the engineer and report the result.",
    description="Executor should always be called after the engineer has written code to be executed.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)

groupchat = GroupChat(
    agents=[user_proxy, engineer, code_executor, writer, planner],
    messages=[],
    max_round=20,
    speaker_selection_method="auto",
)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)


task = f"Today is {datetime.now().date()}. Write a blogpost about the stock price performance of Nvidia in the past month."
print(task)

# Use Cache.disk to cache LLM responses. Change cache_seed for different responses.
with Cache.disk(cache_seed=41) as cache:
    chat_history = user_proxy.initiate_chat(
        manager,
        message=task,
        cache=cache,
    )

Today is 2025-01-14. Write a blogpost about the stock price performance of Nvidia in the past month.
Admin (to chat_manager):

Today is 2025-01-14. Write a blogpost about the stock price performance of Nvidia in the past month.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, we need the following information that can be retrieved using Python code:

1. **Nvidia Stock Price Data**: 
   - Daily closing prices for Nvidia stock over the last month (from December 14, 2024, to January 14, 2025).
   - Opening, high, low prices for each day in that range.

2. **Historical Data Source**: 
   - URL or API endpoint for fetching historical stock price data (e.g., Yahoo Finance, Alpha Vantage, etc.).

3. **Performance Metrics**: 
   - Percentage change in stock price over the past month.
   - Comparison of stock performance with st